In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')
nlp.pipe_names

['tagger', 'parser', 'ner']

In [ ]:
textcat=nlp.create_pipe( "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
nlp.add_pipe(textcat, last=True)
nlp.pipe_names

['tagger', 'parser', 'ner', 'textcat']

In [ ]:
textcat.add_label("POSITIVE")
# textcat.add_label("NEUTRAL")
textcat.add_label("NEGATIVE")

1

In [ ]:
import pandas as pd
reviews = pd.read_csv('https://raw.githubusercontent.com/hanzhang0420/Women-Clothing-E-commerce/master/Womens%20Clothing%20E-Commerce%20Reviews.csv')

# Extract desired columns and view the dataframe 
reviews = reviews[['Review Text','Recommended IND']].dropna()
print(len(reviews))
reviews.head(10)


22641


,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
5,"I love tracy reese dresses, but this one is no...",0
6,I aded this in my basket at hte last mintue to...,1
7,"I ordered this in carbon for store pick up, an...",1
8,I love this dress. i usually get an xs but it ...,1
9,"I'm 5""5' and 125 lbs. i ordered the s petite t...",1


In [ ]:
reviews['tuples'] = reviews.apply(lambda row: (row['Review Text'],row['Recommended IND']), axis=1)
train =reviews['tuples'].tolist()
train[:10]

[('Absolutely wonderful - silky and sexy and comfortable', 1),
 ('Love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.',
  1),
 ('I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c',
  0),
 ("I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear it, i get nothing b

In [ ]:
import random

def load_data(limit=0, split=0.8):
    train_data=train
    # Shuffle the data
    random.shuffle(train_data)
    texts, labels = zip(*train_data)
    # get the categories for each review
    cats = [{"POSITIVE": bool(y), "NEGATIVE": not bool(y)} for y in labels]

    # Splitting the training and evaluation data
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])

n_texts=23486

# Calling the load_data() function 
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(limit=n_texts)

# Processing the final format of training data
train_data = list(zip(train_texts,[{'cats': cats} for cats in train_cats]))
train_data[:10]

[('This is my new favorite top! looks and fits as described.',
  {'cats': {'NEGATIVE': False, 'POSITIVE': True}}),
 ("I fell in love with the shirt when i saw it online; but, i was incredibly disappointed with it when it was delivered.  on the positive side, the macrame is stunning.  once again, there was way too much fabric which makes me look pregnant.  most importantly, the color is not peach!  it's almost a burnt indian red or brick red/orange.  it was a disgusting color.",
  {'cats': {'NEGATIVE': True, 'POSITIVE': False}}),
 ('Purchased this in the wine color and love it! the detail at the neck at the bottom hem gives it an extra flair. i went with a small so that it was more fitted. love that it is stylish and comfortable. paired with knee high boots given it length but could also work well with short boots, like the photo.',
  {'cats': {'NEGATIVE': False, 'POSITIVE': True}}),
 ('I loved this dress on the hanger and loved it more when i tried it on. i am usually an 8, but purchas

In [ ]:
from sklearn.model_selection import train_test_split
X, y = zip(*train)
X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

#### https://www.machinelearningplus.com/nlp/custom-text-classification-spacy/

In [ ]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "NEGATIVE":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}


#("Number of training iterations", "n", int))
n_iter=10

In [ ]:
from spacy.util import minibatch, compounding

# Disabling other components
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()

    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))

    # Performing training
    for i in range(n_iter):
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)

      # Calling the evaluate() function and printing the scores
        with textcat.model.use_params(optimizer.averages):
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'  
              .format(losses['textcat'], scores['textcat_p'],
                      scores['textcat_r'], scores['textcat_f']))

Training the model...
LOSS 	  P  	  R  	  F  
8.589	0.910	0.963	0.936
5.570	0.914	0.963	0.937
4.451	0.917	0.963	0.939
3.446	0.922	0.961	0.941
2.656	0.921	0.961	0.941
2.204	0.923	0.958	0.940
1.728	0.925	0.959	0.941
1.400	0.925	0.956	0.940
1.229	0.925	0.953	0.939
1.079	0.922	0.955	0.938


In [ ]:
test_text = "I loved the dress before buying it now its not okay."
doc = nlp(test_text)
doc.cats

{'NEGATIVE': 0.9925162196159363, 'POSITIVE': 0.0074837785214185715}